In [ ]:
%%capture
!pip install ibm-watsonx-ai==0.2.6
!pip install langchain==0.1.16
!pip install langchain-ibm==0.1.4
!pip install transformers==4.41.2
!pip install huggingface-hub==0.23.4
!pip install sentence-transformers==2.5.1
!pip install chromadb
!pip install wget==3.2
!pip install --upgrade torch --index-url https://download.pytorch.org/whl/cpu

In [ ]:
# !pip list | grep langchain
!pip list | findstr langchain

In [1]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
import wget

In [2]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

file downloaded


In [3]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

In [4]:
# Splitting the document into chunks

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

Created a chunk of size 1624, which is longer than the specified 1000
Created a chunk of size 1885, which is longer than the specified 1000
Created a chunk of size 1903, which is longer than the specified 1000
Created a chunk of size 1729, which is longer than the specified 1000
Created a chunk of size 1678, which is longer than the specified 1000
Created a chunk of size 2032, which is longer than the specified 1000
Created a chunk of size 1894, which is longer than the specified 1000


16


In [5]:
# Embedding and storing

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')

document ingested


In [6]:
# RETRIEVAL
# LLM model construction

# ensure required packages are available
%pip install -q langchain==0.1.16 openai

import os
from langchain.chat_models import ChatOpenAI
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")


llm_model = "nvidia/nemotron-nano-12b-v2-vl:free"  # OpenRouter model

chat = ChatOpenAI(
    model_name=llm_model,
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.0
)

# llm_chat = LLMMathChain(llm=chat)

Note: you may need to restart the kernel to use updated packages.


In [7]:
qa = RetrievalQA.from_chain_type(llm=chat, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "what is mobile policy?"
qa.invoke(query)

{'query': 'what is mobile policy?',
 'result': "A **Mobile Device Policy** (often called a Mobile Phone Policy) is a set of guidelines and rules established by an organization to regulate the appropriate, secure, and responsible use of mobile devices (e.g., smartphones, tablets) in the workplace or for company-related purposes. Its primary goal is to protect sensitive company data, ensure compliance with legal standards, and maintain productivity while allowing flexibility for employees.\n\n### Key Components of a Mobile Device Policy (based on your organization's context):\n1. **Acceptable Use**  \n   - Mobile devices are primarily for **work-related tasks**.  \n   - **Limited personal use** is permitted during non-work hours, provided it does not interfere with job responsibilities.  \n\n2. **Security**  \n   - Employees must **safeguard their devices and access credentials** (e.g., passwords, biometrics).  \n   - Caution is advised when downloading apps or clicking links from untrus

In [ ]:
qa = RetrievalQA.from_chain_type(llm=chat, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

In [8]:
# Using a Prompt Template

prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [9]:
qa = RetrievalQA.from_chain_type(llm=chat, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': 'The provided policies do not explicitly address whether eating is permitted in company vehicles. The only mention of company vehicles is in the **Smoking Policy**, which prohibits smoking in them. Since there is no specific rule against eating in company vehicles, it would be allowed **unless prohibited by other unmentioned guidelines or company-specific rules**. For confirmation, you may need to consult additional company policies or ask your supervisor.\n'}

In [10]:
query = "What I cannot do in it?"
qa.invoke(query)

{'query': 'What I cannot do in it?',
 'result': 'Based on the **Internet and Email Policy** and **Mobile Phone Policy**, the following actions are **prohibited**:\n\n### **Internet and Email Policy Prohibitions:**\n1. **Unauthorized Personal Use**: Using company-provided internet or email for non-work-related activities *during work hours*, unless limited personal use during non-work hours is permitted.\n2. **Sharing Credentials**: Disclosing or sharing login passwords or access credentials.\n3. **Security Risks**: Opening or downloading suspicious email attachments/links from unknown sources without caution.\n4. **Confidentiality Breaches**: Transmitting sensitive company data (e.g., customer information, trade secrets) via unencrypted email or insecure messaging apps.\n5. **Harassment/Inappropriate Content**: Engaging in or distributing content that harasses, discriminates, or offends others via company tools.\n6. **Legal Non-Compliance**: Violating laws related to copyright, data pr

In [11]:
# Adding Conversation Memory

memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)


In [12]:
qa = ConversationalRetrievalChain.from_llm(llm=chat, 
                                           chain_type="stuff", 
                                           retriever=docsearch.as_retriever(), 
                                           memory = memory, 
                                           get_chat_history=lambda h : h, 
                                           return_source_documents=False)

history = []

In [14]:
query = "What is mobile policy?"
result = qa.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

The **Mobile Phone Policy** (as outlined in your organization's guidelines) governs the appropriate and secure use of mobile devices in the workplace. Key components include:  

1. **Acceptable Use**:  
   - Mobile devices should primarily support work-related tasks.  
   - Limited personal usage is permitted, provided it does not interfere with job responsibilities.  

2. **Security**:  
   - Safeguard devices with access controls (e.g., passwords, encryption).  
   - Avoid downloading apps or links from suspicious sources.  
   - Report lost/stolen devices immediately and notify IT/security teams.  

3. **Confidentiality**:  
   - Do not share sensitive company data via unsecured apps or public networks.  
   - Exercise discretion when discussing work matters in public settings.  

4. **Compliance**:  
   - Adhere to laws and regulations regarding data protection, privacy, and device usage.  

5. **Consequences of Non-Compliance**:  
   - Violations may result in disciplinary actions

In [15]:
history.append((query, result["answer"]))

In [16]:
query = "What is the aim of it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

The aim of the Mobile Phone Policy is to ensure employees use mobile devices in a manner that aligns with the organization's values, legal requirements, and security standards. It establishes guidelines for responsible usage (prioritizing work-related tasks), emphasizes safeguarding devices and data, prohibits sharing sensitive information via unsecured channels, and mandates reporting lost or stolen devices. The policy promotes accountability, protects company confidentiality, and complies with data protection laws to maintain a secure and productive work environment.



In [17]:
# Wrap Up and Make an Agent

def qa():
    memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm=chat, 
                                               chain_type="stuff", 
                                               retriever=docsearch.as_retriever(), 
                                               memory = memory, 
                                               get_chat_history=lambda h : h, 
                                               return_source_documents=False)
    history = []
    while True:
        query = input("Question: ")
        
        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break
            
        result = qa({"question": query}, {"chat_history": history})
        
        history.append((query, result["answer"]))
        
        print("Answer: ", result["answer"])

In [18]:
qa()    

Answer:  Understanding what you can and cannot do depends on the specific policies and rules of your organization, workplace, or the context in which you operate (e.g., professional, legal, digital, or personal settings). Since I don’t have access to your unique policies or circumstances, here’s a general framework based on common organizational, legal, or ethical guidelines:

---

### **What You Can Do** (General Guidelines):
1. **Follow Established Protocols**:  
   - Adhere to workplace policies, safety procedures, or legal requirements (e.g., data privacy laws, copyright rules).  
   - Use tools/services (like company email or software) only as permitted by your organization.  

2. **Prioritize Safety and Ethics**:  
   - Act responsibly in environments where your actions could impact others (e.g., workplace, public spaces).  
   - Report unsafe conditions, harassment, or violations of policies.  

3. **Use Digital Tools Responsibly**:  
   - Respect intellectual property (no plagi